In [25]:
def grid(M,N):
    ux = []
    uy = []
    vx = []
    vy = []
    for i in range(M):
            for j in range(N):
                ux.append([])
                uy.append([])
                vx.append([])
                vy.append([])
                ux[i].append(3*j)
                if j % 2 == 0:
                    uy[i].append(2*i)
                else:
                    uy[i].append(2*i + 1)
                vx[i].append(ux[i][j] + 2)
                vx[i].append(ux[i][j])
    return ux, uy, vx, vy

In [27]:
ux[0][0]

NameError: name 'ux' is not defined